# ライントレース+物体検知

In [2]:
import sys
sys.path.append( "/home/jetbot/g031r066/darknet" )
print(sys.path)

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/jetbot/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/torchvision-0.4.0a0+d31eafa-py3.6-linux-aarch64.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_SSD1306-1.6.2-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_MotorHAT-1.4.0-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_GPIO-1.0.4-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/spidev-3.4-py3.6-linux-aarch64.egg', '/usr/local/lib/python3.6/dist-packages/Adafruit_PureIO-1.0.4-py3.6.egg', '/usr/local/lib/python3.6/dist-packages/jetbot-0.4.0-py3.6.egg', '/usr/lib/python3/dist-packages', '/usr/lib/python3.6/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/home/jetbot/.ipython', '/home/jetbot/g031r066/darknet']


## 物体検知側の下準備

In [3]:
import darknet
import darknet_images
import time
"""
load model description and weights from config files
args:
    config_file (str): path to .cfg model file
    data_file (str): path to .data model file
    weights (str): path to weights
returns:
    network: trained model
    class_names
    class_colors
"""
"""
network, class_names, class_colors = darknet.load_network(
    '/home/jetbot/g031r066/darknet/cfg/yolov4-tiny.cfg',  
    '/home/jetbot/g031r066/darknet/cfg/coco.data', 
    '/home/jetbot/g031r066/darknet/weights/yolov4-tiny.weights'
    )


"""
network, class_names, class_colors = darknet.load_network(
        '/home/jetbot/g031r066/darknet/custom/yolov4-tiny-custom.cfg', 
        '/home/jetbot/g031r066/darknet/custom/custom.data',  
        '/home/jetbot/g031r066/darknet/custom/backup/yolov4-tiny-custom_final.weights'
    )

## ライントレース側の下準備

In [4]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model_rf = torchvision.models.resnet18(pretrained=False)
model_rf.fc = torch.nn.Linear(512, 2)
model_rf.load_state_dict(torch.load('/home/jetbot/g031r066/note_nvidia/road_following/best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))
# GPU側に転送
device = torch.device('cuda')
model_rf = model_rf.to(device)
model_rf = model_rf.eval().half()

In [5]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [6]:
# 諸々インポート
from IPython.display import display
import ipywidgets
import traitlets

# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
# 描画
#display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1 2022/09/09 0.15の方が綺麗
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

In [7]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

#display(ipywidgets.HBox([y_slider, speed_slider]))
#display(x_slider, steering_slider)

## 本処理

In [8]:
from jetbot import Camera, bgr8_to_jpeg
import datetime
import time

# カメラ起動
camera = Camera(width=300, height=300, capture_width=1280, capture_height=720)  # デフォルトでは Camera(width=224, height=224, fps=21, capture_width=3280, capture_height=2464)
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

blocked_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
bbox_widget = ipywidgets.Image(format='jpeg', width=300, height=300)
label_widget = ipywidgets.IntText(value=1, description='tracked label')
speed_widget = ipywidgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = ipywidgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')
'''
display(ipywidgets.VBox([
    ipywidgets.HBox([image_widget, blocked_widget),
    label_widget,
    speed_widget,
    turn_gain_widget
]))
'''
width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection

In [9]:
import queue
from IPython.display import clear_output

q = queue.Queue()  # メインスレッド<->サブスレッド で画像をやり取りするためのキューを作っておく
angle = 0.0
angle_last = 0.0
# ライントレースを実装する関数
def road_following(snapshot):
    global angle, angle_last

    xy = model_rf(preprocess(snapshot)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    

# 物体検知を実装する関数
def object_detection(snapshot):
    
    re = 0
    prev_time = time.time()
    thresh = .45
    image, detections = darknet_images.image_detection(snapshot, network, class_names, class_colors, thresh)
    darknet.print_detections(detections, True)  # True: 各物体の座標・幅・高さを表示
    fps = int(1/(time.time() - prev_time))
    print("FPS: {}".format(fps), flush=True)
    print("Predicted in {} seconds".format((time.time() - prev_time)), flush=True)
    
    label_list=[]
    confidence_list=[]
    bbox_list=[]
    for label, confidence, bbox in detections: 
        label_list.append(label)
        confidence_list.append(confidence)
        bbox_list.append(bbox)
    
    if("Traffic light" in label_list):  # 信号機を検知した場合
        # 対象の信号機を1つに絞る
        trafficlight_label_list=[]
        trafficlight_confidence_list=[]
        trafficlight_bbox_list=[]
        for label, confidence, bbox in detections: 
            if(label=="Traffic light"):
                trafficlight_label_list.append(label)
                trafficlight_confidence_list.append(confidence)
                trafficlight_bbox_list.append(bbox)
        
        target_index = trafficlight_confidence_list.index(max(trafficlight_confidence_list))  # 検知した信号機のうち、1番信頼度の高いものを対象にし、そのindexを取ってくる
        target_bbox =  trafficlight_bbox_list[target_index]  # 取ってきたindexを基に、1版信頼度の高い信号機の座標を保存
        print("target_bbox: {}".format(target_bbox), flush=True)
        
        xmin, ymin, xmax, ymax = darknet.bbox2points(target_bbox)
        cropped_image = image[ymin+1:ymax-1, xmin+1:xmax-1]  # bboxの枠線が映らないようにちょっと狭めに切り取る
        
        # 1.HSVに変換して白黒でマスク(2値化)
        # cropped_imageが空になっていることがある、要処理追加
        hsv_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
        # 信号機の目:白,それ以外:黒 でマスクする 
        # 彩度(s)・明度(v)で絞り込む. 信号機の目は大体鮮やかなのでそのようになるように指定
        lower = np.array([0, 200,100])           # 抽出する色の下限(h,s,v)
        upper = np.array([150, 255 , 255])        # 抽出する色の上限(h,s,v)
        mask_traffic_light = cv2.inRange(hsv_cropped_image, lower, upper) # inRangeで元画像を２値化
        # 1の結果を基に、信号機の目部分の色はそのままに、それ以外を黒でマスク
        target = cv2.bitwise_and(hsv_cropped_image,hsv_cropped_image, mask=mask_traffic_light)
        # 信号機の目以外は黒色(H=0)でマスクしている為、Hの平均値を出しても小さい値にしかならないので意味がない
        # → なのでHのminとmaxを出す(min要らないけど)
        h_min = target.T[0].flatten().min()
        h_max = target.T[0].flatten().max()
        
        # Hの最大値で判別する
        if(0 <= h_max <= 30):
            print("Traffic_light : red", flush=True)
            # 赤信号なので一時停止
            robot.left_motor.value = 0.0
            robot.right_motor.value = 0.0
            re = -1
            time.sleep(0.1) 
        elif(30<h_max and h_max<=150):  # 緑と青は取り敢えず一緒にした
            print("Traffic_light : green", flush=True)
            re = 0
        else:
            print("Traffic_light : none", flush=True)
            re = -2
    else:  # 信号機を検知できなかった場合
        now = datetime.datetime.now().isoformat(sep=' ', timespec='milliseconds')
        filename = '/home/jetbot/g031r066/darknet/bad_snapshot/' + str(now) + '.jpg'
        #cv2.imwrite(filename, snapshot)  # 信号機を検知できなかったときのスナップショットを保存
    
    clear_output(True)
            
    q.put(re)
    q.put(image)

In [10]:
import threading

def execute(change):
    snapshot = change['new']  # 映像読み込み type(snapshot) = numpy.ndarray
    
    # 物体検知をサブスレッドで行う
    thread_od = threading.Thread(target = object_detection, args=(snapshot,))
    thread_od.start()
    thread_od.join()
    re = q.get()  # 物体検知の結果を示す返り値(int)を格納
    image = q.get()  # bboxを描画した画像を格納
    
    if(re == -1):  # 赤信号時
        print("road_following don't work.(re == -1)")
    else:
        # 赤信号ではないときのみ、ライントレースを行う
        print("road_following is work.")
        thread_rf = threading.Thread(target = road_following, args=(snapshot,))  # サブスレッドで行う
        thread_rf.start()
        thread_rf.join()
    
    # 出力されているカメラ映像の更新
    bbox_widget.value = bgr8_to_jpeg(image)
    #bbox_widget.value = bgr8_to_jpeg(snapshot)

In [11]:
from ipywidgets import HBox, VBox

display(HBox([bbox_widget, VBox([speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])]))

In [12]:
execute({'new': camera.value})
camera.observe(execute, names='value')

road_following is work.

Objects:
Traffic light: 48.44%    (left_x: 260   top_y:  90   width:   66   height:  201)
FPS: 11
Predicted in 0.08989381790161133 seconds
target_bbox: (260.1303405761719, 89.51848602294922, 65.86913299560547, 200.8131866455078)


Exception in thread Thread-170:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-9-51d5e4c6d685>", line 70, in object_detection
    hsv_cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2HSV)
cv2.error: OpenCV(4.1.1) /home/nvidia/host/build_opencv/nv_opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'




In [27]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing
camera.stop()
robot.stop()

road_following is work.


In [15]:
camera.stop()
robot.stop()

KeyboardInterrupt: 

In [12]:
camera.restart()
time.sleep(0.1)

KeyboardInterrupt: 

In [13]:
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0